In [1]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import time
import pandas as pd

In [2]:
# URL of the website
URL = "https://www.redbus.in/online-booking/astc/?utm_source=rtchometile"
def initialize_driver():
    driver = webdriver.Chrome()
    driver.maximize_window()
    return driver

def load_page(driver, url):
    driver.get(url)
    time.sleep(5)  # Wait for the page to load

In [3]:
def scrape_bus_routes(driver):
    route_elements = driver.find_elements(By.CLASS_NAME, 'route')
    bus_routes_link = [route.get_attribute('href') for route in route_elements]
    bus_routes_name = [route.text.strip() for route in route_elements]
    return bus_routes_link, bus_routes_name
def scrape_bus_details(driver, url, route_name):
    try:
        driver.get(url)
        time.sleep(5)  # Allow the page to load
        
        # Click the "View Buses" button if it exists
        try:
            view_buses_button = WebDriverWait(driver, 10).until(
                EC.element_to_be_clickable((By.CLASS_NAME, "button"))
            )
            driver.execute_script("arguments[0].click();", view_buses_button)
            time.sleep(5)  # Wait for buses to load
            # Scroll down to load all bus items
            driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
            time.sleep(5)  # Wait for the page to load more content

            # Find bus item details
            bus_name_elements = driver.find_elements(By.CLASS_NAME, "travelsName___61bd75")
            bus_type_elements = driver.find_elements(By.CLASS_NAME, "busType___44b170")
            departing_time_elements = driver.find_elements(By.CLASS_NAME, "boardingTime___6bc587")
            duration_elements = driver.find_elements(By.CLASS_NAME, "duration___da3927")
            reaching_time_elements = driver.find_elements(By.CLASS_NAME, "droppingTime___a452f5")
            star_rating_elements = driver.find_elements(By.CLASS_NAME, "rating___f7ba20")
            price_elements = driver.find_elements(By.CLASS_NAME, "finalFare___c05744")

            # Use XPath to handle both seat availability classes
            seat_availability_elements = driver.find_elements(By.XPATH, "//div[contains(@class, 'totalSeats___5f5f18') or contains(@class, 'seatsWrap___1a5212')]")

            bus_details = []
            for i in range(len(bus_name_elements)):
                bus_detail = {
                    "Route_Name": route_name,
                    "Route_Link": url,
                    "Bus_Name": bus_name_elements[i].text,
                    "Bus_Type": bus_type_elements[i].text,
                    "Departing_Time": departing_time_elements[i].text,
                    "Duration": duration_elements[i].text,
                    "Reaching_Time": reaching_time_elements[i].text,
                    "Star_Rating": star_rating_elements[i].text if i < len(star_rating_elements) else '0',
                    "Price": price_elements[i].text,
                    "Seat_Availability": seat_availability_elements[i].text if i < len(seat_availability_elements) else '0'
                }
                bus_details.append(bus_detail)
            return bus_details
        
        except Exception as e:
            print(f"Error occurred while scraping bus details for {url}: {str(e)}")
            return []

    except Exception as e:
        print(f"Error occurred while accessing {url}: {str(e)}")
        return []

# List to hold all bus details
all_bus_details = []

driver = None

# Function to scrape all pages
def scrape_all_pages():
    for page in range(1, 6):  # There are 5 pages
        try:
            driver = initialize_driver()
            load_page(driver, URL)
            
            if page > 1:
                pagination_tab = WebDriverWait(driver, 10).until(
                    EC.presence_of_element_located((By.XPATH, f"//div[contains(@class, 'DC_117_pageTabs')][text()='{page}']"))
                )
                driver.execute_script("arguments[0].scrollIntoView();", pagination_tab)
                driver.execute_script("arguments[0].click();", pagination_tab)
                time.sleep(5)  # Wait for the page to load
            
            all_bus_routes_link, all_bus_routes_name = scrape_bus_routes(driver)
            # Iterate over each bus route link and scrape the details
            for link, name in zip(all_bus_routes_link, all_bus_routes_name):
                bus_details = scrape_bus_details(driver, link, name)
                if bus_details:
                    all_bus_details.extend(bus_details)
            

        except Exception as e:
            print(f"Error occurred while accessing page {page}: {str(e)}")

# Scrape routes and details from all pages
scrape_all_pages()

# Convert the list of dictionaries to a DataFrame
df = pd.DataFrame(all_bus_details)# Close the driver
df.to_csv('assam_bus_details.csv', index=False)
if driver is not None:
    driver.quit()

Error occurred while accessing page 3: Message: 
Stacktrace:
	GetHandleVerifier [0x0x7ff6b2d0fe75+79173]
	GetHandleVerifier [0x0x7ff6b2d0fed0+79264]
	(No symbol) [0x0x7ff6b2ac9e5a]
	(No symbol) [0x0x7ff6b2b20586]
	(No symbol) [0x0x7ff6b2b2083c]
	(No symbol) [0x0x7ff6b2b74247]
	(No symbol) [0x0x7ff6b2b489af]
	(No symbol) [0x0x7ff6b2b7100d]
	(No symbol) [0x0x7ff6b2b48743]
	(No symbol) [0x0x7ff6b2b114c1]
	(No symbol) [0x0x7ff6b2b12253]
	GetHandleVerifier [0x0x7ff6b2fda2ad+3004797]
	GetHandleVerifier [0x0x7ff6b2fd46fd+2981325]
	GetHandleVerifier [0x0x7ff6b2ff3350+3107360]
	GetHandleVerifier [0x0x7ff6b2d2a9fe+188622]
	GetHandleVerifier [0x0x7ff6b2d3228f+219487]
	GetHandleVerifier [0x0x7ff6b2d18dc4+115860]
	GetHandleVerifier [0x0x7ff6b2d18f79+116297]
	GetHandleVerifier [0x0x7ff6b2cff528+11256]
	BaseThreadInitThunk [0x0x7ffa2bb7257d+29]
	RtlUserThreadStart [0x0x7ffa2c54af08+40]

Error occurred while accessing page 4: Message: 
Stacktrace:
	GetHandleVerifier [0x0x7ff6b2d0fe75+79173]
	GetHandle

In [4]:
df

,Route_Name,Route_Link,Bus_Name,Bus_Type,Departing_Time,Duration,Reaching_Time,Star_Rating,Price,Seat_Availability
0,Guwahati to Tezpur,https://www.redbus.in/bus-tickets/guwahati-to-...,Kalpana Travels,AC Seater (2+2),16:50,3h 25m,20:15,4.5,₹351,26 Seats
1,Guwahati to Tezpur,https://www.redbus.in/bus-tickets/guwahati-to-...,Karisma Aarna,A/C Seater Push Back (2+1),17:00,4h 30m,21:30,4.8,₹351,24 Seats
2,Guwahati to Tezpur,https://www.redbus.in/bus-tickets/guwahati-to-...,Shruti(North East Travels),Tata A/C Seater (2+2),17:30,3h 55m,21:25,3.9,₹340,29 Seats
3,Guwahati to Tezpur,https://www.redbus.in/bus-tickets/guwahati-to-...,Yatra Travels,NON AC Seater/ Sleeper (2+1),19:00,4h 45m,23:45,4.6,₹375,23 Seats\n(6 Single)
4,Guwahati to Tezpur,https://www.redbus.in/bus-tickets/guwahati-to-...,Yatra Travels,A/C Seater / Sleeper (2+1),20:00,5h 30m,01:30,3.9,₹450,12 Seats
5,Guwahati to Tezpur,https://www.redbus.in/bus-tickets/guwahati-to-...,Laxmi Travels,A/C Seater (2+1),20:15,4h 55m,01:10,4.8,₹450,13 Seats\n(1 Single)
6,Guwahati to Tezpur,https://www.redbus.in/bus-tickets/guwahati-to-...,Yatra Travels,NON AC Seater/ Sleeper (2+1),20:30,4h 30m,01:00,3.6,₹375,33 Seats\n(2 Single)
7,Guwahati to Tezpur,https://www.redbus.in/bus-tickets/guwahati-to-...,Yatra Travels,NON A/C Seater/ Sleeper (2+1),21:00,4h 30m,01:30,3.6,₹375,27 Seats\n(7 Single)
8,Guwahati to Tezpur,https://www.redbus.in/bus-tickets/guwahati-to-...,MAHARATHI TRANSPORT,Non AC Seater (2+1),21:15,3h 30m,00:45,4.5,₹350,24 Seats\n(6 Single)
9,Guwahati to Tezpur,https://www.redbus.in/bus-tickets/guwahati-to-...,Pranjit Travels (Under ASTC),Bharat Benz A/C Seater (2+1),21:30,5h,02:30,0,₹500,11 Seats
